In [1]:
import numpy as np
from scipy import optimize
import itertools
import math
import random
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')
import time

In [2]:
def k_LP(k,a,eps):
    var_n = int(math.pow(2,k))-1; ub_n = int(math.pow(2,k-1)*k)
    l = np.zeros(k)
    for i in range(k):
        l[i] = int(i)
    p = np.zeros(k)
    p[0] = 1; comb = [0]*(k)
    for i in range(1,k):
        p[i] = p[i-1] + math.comb(k,i)
        comb[i] = list(itertools.combinations(l,i))
    t = np.ones(var_n)
    u = 1
    for h in range(1,k):
        for i in range(int(math.comb(k,h))):
            for j in range(h):
                t[u] *= (a[int(comb[h][i][j])]-1)
            u += 1
    
    A_ub = np.zeros((ub_n,var_n))
    r = 0
    for i in range(k):
        A_ub[r][0] = -1; A_ub[r][i+1] = 1
        r += 1
    g = 1
    for h in range(1,k-1):
        for i in range(int(math.comb(k,h))):
            for j in range(int(math.comb(k,h+1))):
                f = np.zeros(h)
                for z in range(h):
                    for v in range(h+1):
                        if comb[h+1][j][v] == comb[h][i][z]:
                            f[z] = 1
                            break
                if np.sum(f) == h:
                    A_ub[r][g] = -1; A_ub[r][int(p[h])+j] = 1
                    r += 1
            g += 1
    for i in range(k):
        A_ub[r+i][int(p[k-2])+i] = -1
    
    #print(A_ub)
    
    b_ub = np.zeros(ub_n)
    for i in range(k):
        b_ub[ub_n-1-i] = -1
        
    #print(b_ub)
    
    A_eq = np.zeros((k,var_n))
    for s in range(k):
        A_eq[s][0] = 1
        c = 1
        for h in range(1,k):
            for i in range(int(math.comb(k,h))):
                ff = 0
                for j in range(h):
                    if comb[h][i][j] == s:
                        ff = 1
                        break
                if ff == 1:
                    A_eq[s][c] = -(t[c]*math.exp(eps[s]))/(a[s]-1)
                else:
                    A_eq[s][c] = t[c]
                c += 1
    #print(A_eq)
    
    b_eq = np.zeros(k)
    st = 1
    for i in range(k):
        st *= (a[i]-1)
    for s in range(k):
        b_eq[s] = (st*math.exp(eps[s]))/(a[s]-1)
    
    #print(b_eq)
    
    d = np.zeros(var_n); d[0] = 1
    
    ans = optimize.linprog(d,A_ub,b_ub,A_eq,b_eq)
    
    #print(ans.status)
    #print(ans.success)
    #print(ans.message)
        
    return ans

In [3]:
def two_opt(a,eps):
    c = math.exp(eps[0]); d = math.exp(eps[1])
    if c*d >= (a[0]-1)*(a[1]-1) and a[1]*c-a[0]*d+a[0]-a[1] >= 0:
        s0 = (a[1]*c*d+(a[0]-1)*(a[1]-1)*d-(a[0]-1)*(a[1]-1))/(d+a[1]-1)
        s1 = 1
        s2 = (a[1]*c-(a[0]-1)*d+a[0]-1)/(d+a[1]-1)
    elif c*d >= (a[0]-1)*(a[1]-1) and a[1]*c-a[0]*d+a[0]-a[1] < 0:
        s0 = (a[0]*c*d+(a[0]-1)*(a[1]-1)*c-(a[0]-1)*(a[1]-1))/(c+a[0]-1)
        s1 = (-(a[1]-1)*c+a[0]*d+a[1]-1)/(c+a[0]-1)
        s2 = 1
    elif c*d < (a[0]-1)*(a[1]-1) and (a[0]-a[1])*c*d+a[0]*(a[1]-1)*c-(a[0]-1)*a[1]*d >= 0:
        s0 = ((a[0]-1)*c*(d+a[1]-1))/(-c*d+c+(a[0]-1)*a[1])
        s1 = ((a[1]-1)*c*d-(a[1]-1)*c+(a[0]-1)*a[1]*d)/(-c*d+c+(a[0]-1)*a[1])
        s2 = ((a[0]-1)*c*(d+a[1]-1))/(-c*d+c+(a[0]-1)*a[1])
    else:
        s0 = ((a[1]-1)*d*(c+a[0]-1))/(-c*d+d+a[0]*(a[1]-1))
        s1 = ((a[1]-1)*d*(c+a[0]-1))/(-c*d+d+a[0]*(a[1]-1))
        s2 = ((a[0]-1)*c*d+a[0]*(a[1]-1)*c-(a[0]-1)*d)/(-c*d+d+a[0]*(a[1]-1))
    s3 = 1
    return s0,s1,s2

def k_inductive(k,a,eps):
    x = np.zeros((k+1,k+1))
    x[2][0],x[2][1],x[2][2] = two_opt([a[0],a[1]],[eps[0],eps[1]])
    s = a[0]; p = a[0]
    for i in range(3,k+1):
        for j in range(1,i):
            x[i][j] = a[i-1]*x[i-1][j]-a[i-1]+1
        s += a[i-2]; p *= a[i-2]
        s0 = 0
        for j in range(1,i):
            s0 += (a[j-1]-1)*x[i][j]
        left = [[1,a[i-1]-1], [1,-math.exp(eps[i-1])]]
        right = [a[i-1]*x[i-1][0], -s0-p+s-i+2+math.exp(eps[i-1])*(p-1)]
        x[i][0],x[i][i] = np.linalg.solve(left,right)
        f = 0
        if x[i][0] < 1 or x[i][i] < 1:
            f = 1
        for j in range(1,i+1):
            if x[i][0] < x[i][j]:
                f = 1; break
        if f == 1:
            x[i][0] = a[i-1]*x[i-1][0]-a[i-1]+1
            x[i][i] = 1
            eps[i-1] = math.log((x[i][0]+s0+p-s+i-2)/(x[i][i]+p-1))
            
    return x[k],eps

In [4]:
def evaluation(k):
    a = np.zeros(k); eps = np.zeros(k)
    rt = np.zeros(2)
    
    for i in range(k):
        a[i] = int(random.uniform(2,6))
        eps[i] = random.uniform(1,10)
    s = time.time()
    h_ans,eps = k_inductive(k,a,eps)
    e = time.time()
    rt[0] = e-s
    if k <= 10:
        s = time.time()
        opt = k_LP(k,a,eps)
        e = time.time()
        rt[1] = e-s
    
    return rt

In [5]:
# k = 3

rt = np.zeros(2); k = 3
for i in range(10):
    t = evaluation(k)
    print(t)
    rt += t
rt /= 10
print("-------")
print("Heuristic: ", rt[0], "[s]")
print("Optimal:   ", rt[1], "[s]")

[0.00020504 0.00786209]
[7.17639923e-05 7.35521317e-03]
[0.00012589 0.00390315]
[5.31673431e-05 5.26070595e-03]
[5.69820404e-05 8.03995132e-03]
[0.00011897 0.00498676]
[6.67572021e-05 5.87320328e-03]
[6.29425049e-05 7.68303871e-03]
[8.48770142e-05 4.64105606e-03]
[7.29560852e-05 4.83608246e-03]
-------
Heuristic:  9.19342041015625e-05 [s]
Optimal:    0.006044125556945801 [s]


In [6]:
# k = 5

rt = np.zeros(2); k = 5
for i in range(10):
    t = evaluation(k)
    print(t)
    rt += t
rt /= 10
print("-------")
print("Heuristic: ", rt[0], "[s]")
print("Optimal:   ", rt[1], "[s]")

[0.00018096 0.01348424]
[0.00016093 0.010921  ]
[0.00022006 0.01800299]
[0.00016189 0.01208496]
[0.00020409 0.01785684]
[0.0001471  0.01538396]
[0.00017691 0.01332211]
[0.00017309 0.01029086]
[0.00013208 0.01248074]
[0.00012517 0.01109195]
-------
Heuristic:  0.0001682281494140625 [s]
Optimal:    0.013491964340209961 [s]


In [7]:
# k = 100

rt = np.zeros(2); k = 100
for i in range(10):
    t = evaluation(k)
    print(t[0])
    rt += t
rt /= 10
print("-------")
print("Heuristic: ", rt[0], "[s]")

0.014670133590698242
0.019683837890625
0.01949906349182129
0.016455888748168945
0.016125917434692383
0.014491796493530273
0.012253761291503906
0.014777898788452148
0.012420177459716797
0.012483835220336914
-------
Heuristic:  0.01528623104095459 [s]


In [8]:
# k = 500

rt = np.zeros(2); k = 500
for i in range(10):
    t = evaluation(k)
    print(t[0])
    rt += t
rt /= 10
print("-------")
print("Heuristic: ", rt[0], "[s]")

0.2852330207824707
0.24553489685058594
0.27571725845336914
0.27469801902770996
0.2905452251434326
0.28191518783569336
0.2616729736328125
0.303969144821167
0.25128912925720215
0.2618370056152344
-------
Heuristic:  0.2732411861419678 [s]


In [9]:
# k = 1000

rt = np.zeros(2); k = 1000
for i in range(10):
    t = evaluation(k)
    print(t[0])
    rt += t
rt /= 10
print("-------")
print("Heuristic: ", rt[0], "[s]")

1.1987409591674805
1.154487133026123
1.1467278003692627
1.1120631694793701
1.1277430057525635
1.1283552646636963
1.1546380519866943
1.15531587600708
1.1320831775665283
1.130944013595581
-------
Heuristic:  1.144109845161438 [s]
